<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/flask%20server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed May 24 12:46:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

# install package part of model
!pip install simpletransformers
!pip install torch
!pip install pythainlp
!pip install python-crfsuite
!pip install sacremoses

# install package part of flask server
!rm -rf '/content/ngrok-stable-linux-amd64.tgz'
!rm -rf '/content/ngrok'
!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install -U flask-cors
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2Q8ZedCqtQIP2l7e7KVREa2MMvc_2qRULQx4QRLYG97GMTyaZ

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-24 12:47:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP reque

In [3]:
# import part of model
import logging
import torch
import sacremoses
from simpletransformers.t5 import T5Model, T5Args
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# import part of sentence tokenize
import re
import pycrfsuite
from pythainlp import sent_tokenize

# import part of flask server
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok

In [4]:
def split_sentence_TH(TH_predict):
  TH_predict = sent_tokenize(TH_predict)
  for i in range(len(TH_predict)):
    # remove space in back
    TH_predict[i] = TH_predict[i].rstrip()

    # debug for engine fail example ['750ml', '1500ml']
    # move to append previous sentences
    # and replace ''
    if re.search('[0-9]', TH_predict[i][0]):
      j = i
      while True:
        if TH_predict[j-1] != '':
          TH_predict[j-1] += ' ' + TH_predict[i]
          break
      j -= 1
      TH_predict[i] = ''

  # remove '' in list
  while '' in TH_predict:
    TH_predict.remove('')
  return TH_predict

def split_sentence_EN(EN_predict):
  EN_predict = EN_predict.split('. ')
  for i in range(len(EN_predict)):
    if EN_predict[i][-1] != '.':
      EN_predict[i] += '.'
  return EN_predict
  
def predict(machine, text):
  if machine == 'mt5':
    predictions = model_mt5.predict(text)
  elif machine == 'marian':
    predictions = model_marian.predict(text)
  elif machine == 'mbart':
    predictions = model_mbart.predict(text)
  return predictions

In [5]:
app = Flask(__name__) #app name
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)
  
@app.route("/", methods=["GET", "POST"])
def index():
  if request.method == "POST":
    json_data = request.get_json()
    if not json_data:
      return jsonify({"error": "no data"})
    try:
      if json_data["task"] == "th-en":
        prediction = split_sentence_TH(json_data["text"])
      elif json_data["task"] == "en-th":
        prediction = split_sentence_EN(json_data["text"])
      splited_sentence = prediction
      prediction = predict(json_data["machine"], splited_sentence)
      data_predicted = {"prediction": prediction}
      print(splited_sentence)
      print(prediction)
      return jsonify(data_predicted)
    except Exception as e:
      return jsonify({"error": str(e)})
  return "OK"

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


In [ ]:
if __name__ == "__main__":
  # set gpu
  device = torch.cuda.is_available()

  # load mt5
  model_args = T5Args()
  model_args.max_length = 500
  model_args.length_penalty = 1
  model_args.num_beams = 10
  model_mt5 = T5Model("mt5", '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr', args=model_args, use_cuda=device)

  # load marian and mbart
  model_args = Seq2SeqArgs()
  model_args.max_length = 500
  model_args.length_penalty = 1
  model_args.num_beams = 10
  model_marian = Seq2SeqModel(encoder_decoder_type="marian", encoder_decoder_name='/content/gdrive/MyDrive/model/marian-add', args=model_args, use_cuda=device)
  model_mbart = Seq2SeqModel(encoder_decoder_type="mbart", encoder_decoder_name='/content/gdrive/MyDrive/model/mbart-4bs', args=model_args, use_cuda=device)

  # run app
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://fb4b-34-142-179-40.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
